In [0]:
import boto3
import time
import json

In [0]:

try:
    # configuração com o kinesis
    cliente = boto3.client('kinesis',
                         aws_access_key_id='AWS_ACCESS_KEY_ID',
                         aws_secret_access_key='AWS_SECRET_ACCESS_KEY',
                         region_name='sa-east-1')

    # ponto de início da leitura dos dados
    shard = cliente.get_shard_iterator(
               StreamName="stream1",
               ShardId='shardId-000000000002',
               ShardIteratorType='LATEST'
               )["ShardIterator"]

    # loop para consumir os dados
    while shard is not None:
        try:
            resultado = cliente.get_records(ShardIterator=shard)
            registros = resultado['Records']
            shard = resultado["NextShardIterator"]
            
            for registro in registros:
                print("--- Novo Registro ---")
                print(f"SequenceNumber: {registro['SequenceNumber']}")
                print(f"Timestamp: {registro['ApproximateArrivalTimestamp']}")
                print(f"PartitionKey: {registro['PartitionKey']}")
                print(f"Dados: {json.loads(registro['Data'])}")  # converte o json pro dicionário
            
            time.sleep(1)  # esper 1 segundo entre as verificações
            
        except Exception as e:
            print(f"Erro ao consumir dados: {str(e)}")
            time.sleep(5)  # espera 5 segundos pra retomar a leitura
            continue

except Exception as e:
    print(f"Erro na configuração: {str(e)}")   